# Quant Volatility Surface & Pricing Engine Demo

This notebook demonstrates an end-to-end quantitative finance pipeline for:

1.  **Data ETL**: Fetching real-time SPY options data.
2.  **SVI Calibration**: Fitting the Stochastic Volatility Inspired model to market smiles.
3.  **Local Volatility**: Extracting Dupire's Local Volatility surface.
4.  **Exotic Pricing**: Pricing a **Down-and-Out Call Option** using Monte Carlo simulations.
5.  **Hedging Analysis**: Comparing Black-Scholes Delta vs. Local Vol Delta.

In [7]:
import sys
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

# Ensure 'src' is in the python path
if 'src' not in sys.path:
    sys.path.append(os.path.abspath(''))

# Import core project modules
from src.vol_surface import VolatilitySurface
from src.pricer import MonteCarloPricer

print(" Environment setup complete. Project modules loaded.")

 Environment setup complete. Project modules loaded.


## 1. Data ETL & Surface Construction

We initialize the `VolatilitySurface` object. This triggers:
* Fetching live option chain for **SPY** from Yahoo Finance.
* Cleaning data (removing zero-volume/zero-vol quotes).
* Calibrating the **Raw SVI parameters** ($a, b, \rho, m, \sigma$) for every expiration slice.

In [ ]:
TICKER = "SPY"
print(f" Initializing Volatility Surface for {TICKER}...")

# Initialize the engine
surface = VolatilitySurface(TICKER)

# Run the build pipeline (Fetch -> Clean -> Calibrate)
surface.build()

print(f"\n Surface Built Successfully!")
if surface.spot_price:
    print(f" Spot Price: ${surface.spot_price:.2f}")
print(f" Calibrated Expirations: {len(surface.svi_params)} slices")

# Show a sample of the cleaned raw data
print("\n Sample Raw Data (Top 5 rows):")
display(surface.raw_data.head())

 Initializing Volatility Surface for SPY...


2025-12-29 20:18:46,468 - INFO - Current Spot Price for SPY: 690.31
2025-12-29 20:18:46,611 - INFO - Found 30 expiration dates. Starting download...
2025-12-29 20:18:52,169 - INFO - Raw data fetched: 7868 rows.
2025-12-29 20:18:52,194 - INFO - Data cleaning complete. Final dataset size: 95 rows.
2025-12-29 20:18:52,197 - INFO - Building Vol Surface for SPY across 16 expiries.
2025-12-29 20:18:52,259 - INFO - Surface built. Calibrated 6 slices.



 Surface Built Successfully!
 Spot Price: $690.31
 Calibrated Expirations: 6 slices

📋 Sample Raw Data (Top 5 rows):


,contractSymbol,expirationDate,daysToExpiration,T,K,S,moneyness,optionType,bid,ask,mid_price,impliedVolatility,volume,openInterest
837,SPY251231P00740000,2025-12-31,2,0.005479,740.0,690.31,1.071982,put,142.20,145.32,143.760,4.567021,2.0,0.0
2148,SPY260116P00775000,2026-01-16,18,0.049315,775.0,690.31,1.122684,put,273.00,278.00,275.500,3.558503,2.0,0.0
3150,SPY260220P00760000,2026-02-20,53,0.145205,760.0,690.31,1.100955,put,86.01,89.18,87.595,0.416189,2.0,0.0
3189,SPY260227C00589000,2026-02-27,60,0.164384,589.0,690.31,0.853240,call,99.07,102.59,100.830,0.235542,1.0,6.0
3770,SPY260320P00830000,2026-03-20,81,0.221918,830.0,690.31,1.202358,put,220.62,224.38,222.500,0.998978,10.0,0.0


## 2. SVI Calibration Inspection (2D Smile)

Let's inspect a specific maturity slice to see how well the SVI model fits the raw market data.
We look for the characteristic "Smile" or "Skew" shape typical of Equity markets (Left high, Right low).

In [ ]:
# Pick a middle expiration (e.g., closest to 0.5 years or 1 year)
sorted_expirations = sorted(surface.svi_params.keys())
if not sorted_expirations:
    print("No expirations found. Check connection.")
else:
    target_T = sorted_expirations[len(sorted_expirations)//2] # Pick median expiry

    print(f" Inspecting Smile for T = {target_T:.4f} Years")

    # 1. Get Market Data points
    slice_data = surface.raw_data[surface.raw_data['T'] == target_T]
    k_market = np.log(slice_data['moneyness'])
    vol_market = slice_data['impliedVolatility']

    # 2. Generate Model Curve
    k_grid = np.linspace(k_market.min()-0.1, k_market.max()+0.1, 100)
    vol_model = [surface.get_implied_vol(k, target_T) for k in k_grid]
    m_grid = np.exp(k_grid) # Convert log-moneyness back to Strike/Spot ratio

    # 3. Plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=slice_data['moneyness'], y=vol_market, mode='markers', name='Market Data', marker=dict(color='red', size=8, symbol='x')))
    fig.add_trace(go.Scatter(x=m_grid, y=vol_model, mode='lines', name='SVI Model', line=dict(color='blue', width=3)))

    fig.update_layout(
        title=f"SVI Calibration: {TICKER} (T={target_T:.2f} Years)",
        xaxis_title="Moneyness (Strike / Spot)",
        yaxis_title="Implied Volatility",
        template="plotly_white",
        height=500
    )
    fig.show()

🔍 Inspecting Smile for T = 1.0466 Years


## 3. Implied Volatility Surface (3D)

We interpolate the Total Variance ($w = \sigma^2 T$) in the time dimension to create a dense grid.
This represents the **Market's Average Expectation** of volatility over the life of the option.

In [10]:
# Get mesh grid coordinates
X, Y, Z_imp, Z_loc = surface.get_mesh_grid()

fig_imp = go.Figure(data=[go.Surface(z=Z_imp, x=X, y=Y, colorscale='Viridis')])

fig_imp.update_layout(
    title=f'{TICKER} Implied Volatility Surface',
    scene=dict(
        xaxis_title='Moneyness (K/S)',
        yaxis_title='Time to Maturity (Years)',
        zaxis_title='Implied Vol'
    ),
    width=900, height=600,
    margin=dict(l=65, r=50, b=65, t=90)
)
fig_imp.show()

## 4. Local Volatility Surface (Dupire)

Using **Dupire's Formula**, we extract the *instantaneous* volatility $\sigma_{loc}(S, t)$.

Notice how the Local Volatility surface is much **steeper and spikier** than the Implied Volatility surface. 
It amplifies the skew to mathematically explain the market prices for path-dependent pricing.

In [11]:
fig_loc = go.Figure(data=[go.Surface(z=Z_loc, x=X, y=Y, colorscale='Turbo')])

fig_loc.update_layout(
    title=f'{TICKER} Local Volatility Surface (Dupire)',
    scene=dict(
        xaxis_title='Moneyness (Spot/Strike)',
        yaxis_title='Time to Maturity (Years)',
        zaxis_title='Local Vol'
    ),
    width=900, height=600,
    margin=dict(l=65, r=50, b=65, t=90)
)
fig_loc.show()

## 5. Exotic Pricing: Down-and-Out Call

We price a **Down-and-Out Call Option**.
* **Structure:** If the asset price drops below the Barrier, the option becomes worthless.
* **Model Comparison:**
    * **Black-Scholes:** Assumes constant volatility (ATM).
    * **Local Volatility:** Uses the dynamic surface derived above.

**Hypothesis:** Local Volatility should result in a lower price (or different risk profile) because it captures the **Skew** (higher volatility on the downside increases knock-out probability).

In [12]:
# Parameters
if surface.spot_price:
    S0 = surface.spot_price
    K = float(int(S0 * 1.05))       # Strike 5% OTM
    Barrier = float(int(S0 * 0.85)) # Barrier 15% below Spot
    T = 1.0             # 1 Year maturity
    r = 0.045           # Risk-free rate
    N_SIMS = 5000       # Monte Carlo paths

    print(f" Pricing Parameters:")
    print(f"   Spot: ${S0:.2f} | Strike: ${K:.2f} | Barrier: ${Barrier:.2f}")

    # Initialize Pricer
    pricer = MonteCarloPricer(S0, r, T, surface)

    # 1. Black-Scholes Price (Benchmark)
    atm_vol = surface.get_implied_vol(0, T) # k=0 is ATM
    res_bs = pricer.price_barrier_option(K, Barrier, model="black_scholes", const_vol=atm_vol, n_paths=N_SIMS)

    # 2. Local Vol Price
    res_lv = pricer.price_barrier_option(K, Barrier, model="local_vol", n_paths=N_SIMS)

    # Output
    print("\n Pricing Results:")
    print("-" * 40)
    print(f"ATM Volatility Used:   {atm_vol:.2%}")
    print(f"Black-Scholes Price:   ${res_bs['price']:.2f}")
    print(f"Local Vol Price:       ${res_lv['price']:.2f}")
    print("-" * 40)
    print(f"Difference (Model Risk): ${res_lv['price'] - res_bs['price']:.2f}")
else:
    print("Error: Spot price not available.")

 Pricing Parameters:
   Spot: $690.31 | Strike: $724.00 | Barrier: $586.00

 Pricing Results:
----------------------------------------
ATM Volatility Used:   32.64%
Black-Scholes Price:   $77.48
Local Vol Price:       $92.67
----------------------------------------
Difference (Model Risk): $15.19


## 6. Hedging Analysis: Delta Profile

We calculate the **Delta (Hedge Ratio)** across a range of spot prices as the price approaches the Barrier.

This visualization reveals **Model Risk**:
* **Black-Scholes Delta (Gray):** Often underestimates the risk of the barrier.
* **Local Vol Delta (Red):** Adapts to the increasing volatility as spot drops, suggesting a different hedging strategy.

In [ ]:
print(" Calculating Delta Profile ")

if surface.spot_price:
    # Generate Spot Range (From Barrier to ITM)
    spot_range = np.linspace(Barrier * 0.95, S0 * 1.1, 15)
    bs_deltas = []
    lv_deltas = []

    for s_val in spot_range:
        # Temp pricer for hypothetical spot
        temp_pricer = MonteCarloPricer(s_val, r, T, surface)
        
        # Calculate Deltas (Reduced paths for speed in demo)
        d_bs = temp_pricer.calculate_delta(K, Barrier, model="black_scholes", n_paths=2000)
        d_lv = temp_pricer.calculate_delta(K, Barrier, model="local_vol", n_paths=2000)
        
        bs_deltas.append(d_bs)
        lv_deltas.append(d_lv)

    # Plot
    fig_delta = go.Figure()
    fig_delta.add_trace(go.Scatter(x=spot_range, y=bs_deltas, mode='lines+markers', name='BS Delta', line=dict(color='gray', dash='dash')))
    fig_delta.add_trace(go.Scatter(x=spot_range, y=lv_deltas, mode='lines+markers', name='Local Vol Delta', line=dict(color='red', width=3)))
    fig_delta.add_vline(x=Barrier, line_width=2, line_dash="dot", line_color="black", annotation_text="Barrier")

    fig_delta.update_layout(
        title="Delta Skew: Hedging Ratio vs. Spot Price",
        xaxis_title="Spot Price",
        yaxis_title="Option Delta",
        template="plotly_white"
    )
    fig_delta.show()

 Calculating Delta Profile 
